In [22]:
import numpy as np
import tensorflow as tf
import tflearn
import random
from tensorflow.keras.models import load_model
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmeer=LancasterStemmer()

import json
import pickle
import matplotlib.pyplot as plt

In [2]:
print("Processing the intents in data")
with open('/media/revanth/Data/NLP_R/archive/Intent.json') as content2:
    data2=json.load(content2)

Processing the intents in data


Looping through the Intents to convert them the words,classes documents and the ignore_words

In [3]:
words=[]
classes=[]
documents=[]
ignore_words=['?','!']
#print("Looping through the Intents to convert them the words,classes documents and the ignore_words")
for intent in data2['intents']:
    for pattern in intent['text']:
        w=nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w,intent['intent']))
        
        if intent['intent'] not in classes:
            classes.append(intent['intent'])

Stemming ,lowering and removing duplicates

In [4]:
words=[stemmeer.stem(w.lower()) for w in words if w not in ignore_words]
words=sorted(list(set(words)))
#removing the duplicates
classes=sorted(list(set(classes)))

print(len(documents),"documents")
print(len(classes),"classes",classes)
print(len(words)," unique stemmed words",words)

143 documents
22 classes ['Clever', 'CourtesyGoodBye', 'CourtesyGreeting', 'CourtesyGreetingResponse', 'CurrentHumanQuery', 'GoodBye', 'Gossip', 'Greeting', 'GreetingResponse', 'Jokes', 'NameQuery', 'NotTalking2U', 'PodBayDoor', 'PodBayDoorResponse', 'RealNameQuery', 'SelfAware', 'Shutup', 'Swearing', 'Thanks', 'TimeQuery', 'UnderstandQuery', 'WhoAmI']
115  unique stemmed words ["'s", ',', 'a', 'about', 'adam', 'adio', 'am', 'any', 'anyon', 'ar', 'aw', 'bay', 'be', 'bell', 'bor', 'by', 'bye', 'cal', 'camer', 'can', 'che', 'clev', 'commun', 'comprendo', 'conscy', 'could', 'do', 'doing', 'door', 'enough', 'for', 'friend', 'fuck', 'geny', 'get', 'girl', 'giv', 'good', 'goodby', 'gossip', 'got', 'gre', 'hav', 'hear', 'hello', 'help', 'hi', 'hol', 'hop', 'how', 'hya', 'i', 'ident', 'in', 'intellig', 'is', 'it', 'jok', 'know', 'lat', 'laugh', 'mak', 'me', 'mean', 'meant', 'mor', 'my', "n't", 'nam', 'nee', 'not', 'off', 'ok', 'op', 'pleas', 'pod', 'prov', 'quiet', 'real', 'say', 'see', 'self'

Creating a data for our model

In [5]:
training=[]
output=[]
output_empty=[0]*len(classes)

for doc in documents:
    #initialize our bag of words
    bag=[]
    #list the tokenized words
    pattern_words=doc[0]
    pattern_words=[stemmeer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    output_row=list(output_empty)
    output_row[classes.index(doc[1])]=1
    training.append([bag,output_row])

Shuffling randomly and converting into numpy array for faster processing

In [6]:
random.shuffle(training)
training=np.array(training)

print("Creating train and test lists...")
X_train=list(training[:,0])
Y_train=list(training[:,1])
print("buliding neural network for our data chatbot")
tf.compat.v1.reset_default_graph()

Creating train and test lists...
buliding neural network for our data chatbot


/tmp/ipykernel_6760/409062772.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training=np.array(training)


Training

In [7]:
net=tflearn.input_data(shape=[None,len(X_train[0])])
net=tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(Y_train[0]), activation='softmax')
net = tflearn.regression(net)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [8]:
model=tflearn.DNN(net,tensorboard_dir='tflearn_logs')

2023-02-25 15:38:35.360573: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-02-25 15:38:35.360596: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-25 15:38:35.360619: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (kali): /proc/driver/nvidia/version does not exist
2023-02-25 15:38:35.360821: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-25 15:38:35.612833: I tensorflow/compiler/mlir/mlir_graph_optimization_

In [9]:
Train_1=model.fit(X_train,Y_train,n_epoch=600,batch_size=8,show_metric=True)

Training Step: 10799  | total loss: 0.03251 | time: 0.041s
| Adam | epoch: 600 | loss: 0.03251 - acc: 0.9986 -- iter: 136/143
Training Step: 10800  | total loss: 0.02945 | time: 0.043s
| Adam | epoch: 600 | loss: 0.02945 - acc: 0.9987 -- iter: 143/143
--


In [10]:
model.save('model.tflearn')

INFO:tensorflow:/media/revanth/Data/NLP_R/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [11]:
model.save('chat2.h5')

INFO:tensorflow:/media/revanth/Data/NLP_R/chat2.h5 is not in all_model_checkpoint_paths. Manually adding it.


Save the pickle 

In [12]:
print("Pickle is also Saved..........")
pickle.dump( {'words1':words, 'classes1':classes, 'train_x':X_train, 'train_y':Y_train}, open( "training_data", "wb" ) )

Pickle is also Saved..........


In [13]:
print("Loading Pickle.....")
data = pickle.load( open( "training_data", "rb" ) )
words = data['words1']
classes = data['classes1']
train_x = data['train_x']
train_y = data['train_y']


Loading Pickle.....


In [14]:
with open('/media/revanth/Data/NLP_R/archive/Intent.json') as content2:
    data2=json.load(content2)

In [15]:
model1=model.load('model.tflearn')

INFO:tensorflow:Restoring parameters from /media/revanth/Data/NLP_R/model.tflearn


In [16]:
model2=model.load('chat2.h5')

INFO:tensorflow:Restoring parameters from /media/revanth/Data/NLP_R/chat2.h5


In [17]:
def clean_up_sentence(sentence):
    sentence_words=nltk.word_tokenize(sentence)
    
    sentence_words=[stemmeer.stem(word.lower()) for word in sentence_words]
    return sentence_words

In [18]:
def bow(sentence,words,show_details=False):
    sentence_words=clean_up_sentence(sentence)
    bag=[0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w==s:
                bag[i]=1
                if show_details:
                    print("found in bag %" % w)
    return (np.array(bag))
ERROR_THRESHOLD = 0.25
print("ERROR_THRESHOLD = 0.25")

ERROR_THRESHOLD = 0.25


In [19]:
def classify(sentence):
    # Prediction or To Get the Posibility or Probability from the Model
    results = model.predict([bow(sentence, words)])[0]
    # Exclude those results which are Below Threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # Sorting is Done because heigher Confidence Answer comes first.
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1])) #Tuppl -> Intent and Probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # That Means if Classification is Done then Find the Matching Tag.
    if results:
        # Long Loop to get the Result.
        while results:
            for i in data2['intents']:
                # Tag Finding
                if i['intent'] == results[0][0]:
                    # Random Response from High Order Probabilities
                    return print("🤖 BoT :",random.choice(i['responses']))

            results.pop(0)

In [21]:
while True:
    input_data = input("👨‍🦰 You- ")
    answer=response(input_data)
    if input_data=="goodbye":
        break
    

👨‍🦰 You- What is your name
🤖 BoT : Call me Geni
👨‍🦰 You- HI
🤖 BoT : Hello human, please tell me your GeniSys user
👨‍🦰 You- open
🤖 BoT : Jim, I just don't have the power
👨‍🦰 You- ok
🤖 BoT : Any time!
👨‍🦰 You- sure
🤖 BoT : It's life Jim but not as we know it!
👨‍🦰 You- how are you
🤖 BoT : Hello, how are you? I am great thanks! Please tell me your GeniSys user
👨‍🦰 You- i am
🤖 BoT : Hello human, please tell me your GeniSys user
👨‍🦰 You- who am i
🤖 BoT : You may call me Geni
👨‍🦰 You- plz open
🤖 BoT : System says no!
👨‍🦰 You- ok
🤖 BoT : No problem!
👨‍🦰 You- what do you want
🤖 BoT : You can call me Geni
👨‍🦰 You- goodbye
🤖 BoT : Bye! Come back again soon.
